In [1]:
import geopandas as gpd
import pandas as pd
import json

In [2]:
HD33_MUNICIPALITIES = [
    'EAST DEER TWP',
    'FAWN TWP',
    'FRAZER TWP',
    'HARMAR TWP',
    'HARRISON TWP',
    'INDIANA TWP',
    'OHARA TWP',
    'SPRINGDALE TWP',
    'ASPINWALL BORO',
    'BLAWNOX BORO',
    'BRACKENRIDGE BORO',
    'CHESWICK BORO',
    'FOX CHAPEL BORO',
    'SHARPSBURG BORO',
    'SPRINGDALE BORO',
    'TARENTUM BORO'
]

MUNI_TYPES = {
    4: 'TWP',
    6: 'BORO'
}

# Generate GeoJSON map

### Standardize township class, limit municipalities to those in the district

In [18]:
munis = gpd.read_file('./input/municipalities-pa.geojson')

munis.loc[munis.CLASS_OF_M.isin(['1TWP', '2TWP']), 'CLASS_OF_M'] = 'TWP'
munis['muni_name'] = munis.MUNICIPAL1 + ' ' + munis.CLASS_OF_M

munis = munis[(munis.COUNTY == '02') & (munis.muni_name.isin(HD33_MUNICIPALITIES))]
munis.to_file('./output/municipalities-hd33.geojson', driver='GeoJSON')

# Generate presidential election results by municipality

In [38]:
def read_results_csv(filename):
    results = pd.read_csv(filename,
        names=['county', 'office', 'party', 'vote', 'muni_type', 'muni_name'],
        usecols=[2, 8, 9, 15, 19, 20])

    # Add muni type to name
    results = results.replace({ 'O HARA': 'OHARA' })
    results['muni_name'] = results['muni_name'] + ' ' + results['muni_type'].map(MUNI_TYPES)

    return results

def filter_results_to_district_munis(results):
    return results[
        (results.office == 'USP') &
        (results.county == 2) &
        (results.muni_name.isin(HD33_MUNICIPALITIES))
    ]

def sum_results_by_muni(results):
    results_by_muni = pd.pivot_table(results, aggfunc='sum', columns='party', index='muni_name', values='vote')
    results_by_muni = results_by_muni.fillna(0)

    results_by_muni['_SUM'] = results_by_muni.sum(axis=1)
    results_by_muni['DEM_pct'] = round(100 * (results_by_muni.DEM / results_by_muni._SUM), 1)
    results_by_muni['REP_pct'] = round(100 * (results_by_muni.REP / results_by_muni._SUM), 1)
    results_by_muni['OTH_pct'] = round(100 * ((results_by_muni._SUM - results_by_muni.REP - results_by_muni.DEM) / results_by_muni._SUM), 1)

    return results_by_muni[['DEM_pct', 'REP_pct', 'OTH_pct']]

### Process 2012 results

In [39]:
results_2012 = read_results_csv('./input/results_2012_DOS.csv')
results_2012 = filter_results_to_district_munis(results_2012)
results_2012 = sum_results_by_muni(results_2012)
results_2012.head(1)

party,DEM_pct,REP_pct,OTH_pct
muni_name,,,
ASPINWALL BORO,54.3,43.6,2.0


### Process 2020 results

In [43]:
# Read in raw results from DRA, limit to district
results_2020 = pd.read_csv('./input/results_2020_DRA.csv', usecols=['GEOID20', 'District', 'Total_2020_Pres', 'Dem_2020_Pres', 'Rep_2020_Pres'])
results_2020 = results_2020[results_2020.District == 33]

# Merge in VTD data as way to get at the municipality
vtd = gpd.read_file('./input/allegheny_vtd/allegheny_vtd.shp')
vtd.loc[vtd['GEOID20'] == '4200300F585', 'NAME20'] = 'SPRINGDALE TWP' # Add "TWP" for one-precinct Springdale Twp, to not confuse with BORO
results_2020 = pd.merge(results_2020, vtd[['GEOID20', 'NAME20']], on='GEOID20')

# Remove VTD artifacts from municipality name
results_2020['muni_name'] = results_2020.NAME20.str.replace(r' DISTRICT.*', '', regex=True)
results_2020['muni_name'] = results_2020.muni_name.str.replace(r' WARD.*', '', regex=True)
results_2020 = results_2020.replace({ "O'HARA": 'OHARA' })

# Sum results by municipality
results_2020 = pd.pivot_table(results_2020, aggfunc='sum', index='muni_name', values=['Total_2020_Pres', 'Dem_2020_Pres', 'Rep_2020_Pres']).reset_index()

# Use 2012 results from DOS to properly format municipalities in 2020 results from DRA
munis_2012 = pd.read_csv('./input/results_2012_DOS.csv', names=['muni_type', 'muni_name'], usecols=[19, 20]).drop_duplicates()
munis_2012 = munis_2012.replace({ 'O HARA': 'OHARA' })
munis_2012 = munis_2012[munis_2012.muni_name.isin(results_2020.muni_name.unique())]
munis_2012['formatted_muni_name'] = munis_2012['muni_name'] + ' ' + munis_2012['muni_type'].map(MUNI_TYPES)
results_2020 = pd.merge(results_2020, munis[['formatted_muni_name', 'muni_name']], on='muni_name')

# Calculate percentages
results_2020['DEM_pct'] = round(100 * (results_2020.Dem_2020_Pres / results_2020.Total_2020_Pres), 1)
results_2020['REP_pct'] = round(100 * (results_2020.Rep_2020_Pres / results_2020.Total_2020_Pres), 1)
results_2020['OTH_pct'] = round(100 * ((results_2020.Total_2020_Pres - results_2020.Rep_2020_Pres - results_2020.Dem_2020_Pres) / results_2020.Total_2020_Pres), 1)

# Limit columns
results_2020 = results_2020[['formatted_muni_name', 'DEM_pct', 'REP_pct', 'OTH_pct']]
results_2020 = results_2020.rename(columns={ 'formatted_muni_name': 'muni_name' })
results_2020.head(1)

,muni_name,DEM_pct,REP_pct,OTH_pct
0,ASPINWALL BORO,69.7,29.2,1.1


### Merge, compare 2012 and 2020 results

In [45]:
diff_2012_2020 = pd.merge(results_2012, results_2020, on='muni_name', suffixes=['_2012', '_2020'])
diff_2012_2020['DEM_pct_diff'] = round(diff_2012_2020.DEM_pct_2020 - diff_2012_2020.DEM_pct_2012, 1)
diff_2012_2020.head(1)

,muni_name,DEM_pct_2012,REP_pct_2012,OTH_pct_2012,DEM_pct_2020,REP_pct_2020,OTH_pct_2020,DEM_pct_diff
0,ASPINWALL BORO,54.3,43.6,2.0,69.7,29.2,1.1,15.4


### Dump results comparison into CSV file

In [46]:
diff_2012_2020.to_csv('./output/diff_2012_2020.csv', index=False)